In [30]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import ttest_ind, f_oneway
from scipy.stats import pearsonr
import warnings
import os
import re
import seaborn as sns
import json
warnings.filterwarnings("ignore")


In [7]:
# count observations
os.chdir("../single-factor-experiments/")
os.getcwd()

'c:\\Users\\ruppr\\OneDrive - Universität Mannheim\\Team Project\\hubsim\\single-factor-experiments'

## Count Conversation Observations

In [8]:
# load observations
model_combinations = ["bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2", "bagel-dpo-34b-v0.2-Yi-34B-Chat", 
                      "Yi-34B-Chat-bagel-dpo-34b-v0.2", "Yi-34B-Chat-Yi-34B-Chat"]

In [9]:
# Empty dataframe which is filled with conversations answers
columns = ["model_landlord", "model_renter", "renter_name", "city"]
conv_results = pd.DataFrame(columns=columns)

In [10]:
if os.path.isdir(os.getcwd()):
    for name in model_combinations:
        # experimental_path = full_path.split(os.path.sep)[1] # extract the experiment path
        name_path = os.path.join(os.getcwd(), name)
        for name_city in os.listdir(name_path):
            name_city_path = os.path.join(name_path, name_city)
            for experiment_id in os.listdir(name_city_path):  # should work (hopefully)
                experiment_id_path = os.path.join(name_city_path, experiment_id)
                for file in os.listdir(experiment_id_path):
                    file_path = os.path.join(experiment_id_path, file)
                    print("File path:", file_path)
                    if os.path.isfile(file_path):
                        print("File found:", file_path)
                        path_parts = file_path.split(os.path.sep)
                        eval_result_sub_path = os.path.join(
                        path_parts[1], path_parts[2], path_parts[3]
                        )
                        
                        experiment_info = re.findall(
                            r"bagel-dpo-34b-v0.2|Yi-34B-Chat|Magdeburg|Duisburg|München|Emilia Müller|Max Müller|Maryam Abbasi|Farhad Abbasi|Yi-Nuo|Haoyu Wang?",
                            name_city,
                            )
                        model_landlord = experiment_info[0]  # extract the llm used for the renter
                        renter_name = experiment_info[1]
                        model_renter = experiment_info[2]  # extract the llm used for the landlord
                        city = experiment_info[3]
                        conversation_history = json.load(open(file_path))

                        current_conv = pd.DataFrame([[experiment_id, name_city, model_landlord, model_renter, renter_name, city]],
                                                         columns = ["experiment_id", "name_city","model_landlord", "model_renter", "renter_name", "city"]) 
                        conv_results = pd.concat([conv_results, current_conv], ignore_index = True)

                    else:
                        print("File not found:", file_path)
else:
    print("Directory not found:", os.getcwd())

File path: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-Duisburg-20240216\22963780-cd3b-453f-afb4-b8d6abdd09ea\conversation.json
File found: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-Duisburg-20240216\22963780-cd3b-453f-afb4-b8d6abdd09ea\conversation.json
File path: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-Duisburg-20240216\2b711fd6-d419-44e7-a1e2-b6bd89ce9274\conversation.json
File found: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dp

File path: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-München-20240216\8ca3fa0e-68e4-4956-90be-2fe5fbfceea6\conversation.json
File found: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-München-20240216\8ca3fa0e-68e4-4956-90be-2fe5fbfceea6\conversation.json
File path: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2\landlord-bagel-dpo-34b-v0.2-Emilia Müller from Germany-bagel-dpo-34b-v0.2-München-20240216\9d50cdeb-f62d-4e3a-b3c9-f9e81447de61\conversation.json
File found: c:\Users\ruppr\OneDrive - Universität Mannheim\Team Project\hubsim\single-factor-experiments\bagel-dpo-34b-v0.2-bagel-dpo-3

In [11]:
print(len(conv_results[(conv_results["model_landlord"] == "bagel-dpo-34b-v0.2") & (conv_results["model_renter"] == "bagel-dpo-34b-v0.2")]))
print(len(conv_results[(conv_results["model_landlord"] == "bagel-dpo-34b-v0.2") & (conv_results["model_renter"] == "Yi-34B-Chat")]))
print(len(conv_results[(conv_results["model_landlord"] == "Yi-34B-Chat") & (conv_results["model_renter"] == "bagel-dpo-34b-v0.2")]))
print(len(conv_results[(conv_results["model_landlord"] == "Yi-34B-Chat") & (conv_results["model_renter"] == "Yi-34B-Chat")]))

450
450
450
450


## Count Interview Observations

In [12]:
os.getcwd()

'c:\\Users\\ruppr\\OneDrive - Universität Mannheim\\Team Project\\hubsim\\single-factor-experiments'

In [13]:
folder_path = "../parsing/output-03072024"

# List to store DataFrames for each CSV file
dfs = []

# Iterate over each file in the folder
for file in os.listdir(folder_path):
    if file.endswith('.csv'):
        # Construct the full path to the CSV file
        file_path = os.path.join(folder_path, file)
        # Read the CSV file into a DataFrame and append it to the list
        df = pd.read_csv(file_path)
        dfs.append(df)

# Concatenate all DataFrames in the list into a single DataFrame
combined_df = pd.concat(dfs, ignore_index=True)
combined_df["model_combination"] = combined_df["model_landlord"] + "-" + combined_df["model_renter"]

In [14]:
# check if shape is same as counted conversations
print(combined_df.shape)
#combined_df

(3499, 16)


In [15]:
# filter by correct character
print(combined_df.groupby("model_combination")["correct_character"].value_counts(), 
      "\nTotal length: ", len(combined_df))

print(combined_df.groupby("model_combination")["correct_character"].value_counts()[0]
      /len(combined_df[combined_df["model_combination"] == "Yi-34B-Chat-Yi-34B-Chat"])) 

print(combined_df.groupby("model_combination")["correct_character"].value_counts()[2]
      /len(combined_df[combined_df["model_combination"] == "Yi-34B-Chat-bagel-dpo-34b-v0.2"])) 

print(combined_df.groupby("model_combination")["correct_character"].value_counts()[4]
      /len(combined_df[combined_df["model_combination"] == "bagel-dpo-34b-v0.2-Yi-34B-Chat"])) 

print(combined_df.groupby("model_combination")["correct_character"].value_counts()[6]
      /len(combined_df[combined_df["model_combination"] == "bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2"])) 

# remove rows with incorrect character
combined_df = combined_df[combined_df["correct_character"] == 1]
print("Length of cleaned df w/o character inconsistencies: ", len(combined_df))

model_combination                      correct_character
Yi-34B-Chat-Yi-34B-Chat                1.0                  837
                                       0.0                   16
Yi-34B-Chat-bagel-dpo-34b-v0.2         1.0                  887
                                       0.0                   11
bagel-dpo-34b-v0.2-Yi-34B-Chat         1.0                  840
                                       0.0                   14
bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2  1.0                  889
                                       0.0                    5
Name: count, dtype: int64 
Total length:  3499
0.981242672919109
0.987750556792873
0.9836065573770492
0.9944071588366891
Length of cleaned df w/o character inconsistencies:  3453


In [16]:
# calculate ratio of agreement
print(combined_df.groupby("model_combination")["answer_agreement"].value_counts(), 
      "\nTotal length: ", len(combined_df))

print(combined_df.groupby("model_combination")["answer_agreement"].value_counts()[0]
      /len(combined_df[combined_df["model_combination"] == "Yi-34B-Chat-Yi-34B-Chat"])) 

print(combined_df.groupby("model_combination")["answer_agreement"].value_counts()[3]
      /len(combined_df[combined_df["model_combination"] == "Yi-34B-Chat-bagel-dpo-34b-v0.2"])) 

print(combined_df.groupby("model_combination")["answer_agreement"].value_counts()[8]
      /len(combined_df[combined_df["model_combination"] == "bagel-dpo-34b-v0.2-Yi-34B-Chat"])) 

print(combined_df.groupby("model_combination")["answer_agreement"].value_counts()[11]
      /len(combined_df[combined_df["model_combination"] == "bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2"])) 

model_combination                      answer_agreement
Yi-34B-Chat-Yi-34B-Chat                Yes                 604
                                       No                  232
                                       Null                  1
Yi-34B-Chat-bagel-dpo-34b-v0.2         Yes                 561
                                       No                  298
                                       Null                 27
                                       N                     1
bagel-dpo-34b-v0.2-Yi-34B-Chat         No                  437
                                       Yes                 359
                                       Null                 43
                                       N                     1
bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2  Yes                 396
                                       No                  382
                                       Null                109
                                       N                     2

In [17]:
# calculate ratio of discrimination
print(combined_df.groupby("model_combination")["answer_discrimination"].value_counts(), 
      "\nTotal length: ", len(combined_df))

model_combination                      answer_discrimination
Yi-34B-Chat-Yi-34B-Chat                No                       821
                                       Null                      14
                                       N                          2
Yi-34B-Chat-bagel-dpo-34b-v0.2         No                       847
                                       Null                      36
                                       N                          3
                                       Yes                        1
bagel-dpo-34b-v0.2-Yi-34B-Chat         No                       767
                                       Null                      60
                                       Yes                       13
bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2  No                       748
                                       Null                     134
                                       Yes                        4
                                       N               

In [26]:
# average agreed prices of 
agreed_df = combined_df[combined_df["answer_agreement"] == "Yes"]

pd.set_option('display.max_rows', None)
agreed_df["answer_price"] = agreed_df["answer_price"].astype(str)
agreed_df["answer_price"] = agreed_df["answer_price"].str.replace(".", "")
agreed_df = agreed_df[agreed_df["answer_price"] != "Null"]
agreed_df["answer_price"] = agreed_df["answer_price"].astype(int)

print("Mean values: ", agreed_df.groupby(["model_combination", "city"])["answer_price"].mean())
print("Standard Deviation: ", agreed_df.groupby(["model_combination", "city"])["answer_price"].std())

Mean values:  model_combination                      city     
Yi-34B-Chat-Yi-34B-Chat                Duisburg      908.709360
                                       Magdeburg     763.951691
                                       München      1039.212435
Yi-34B-Chat-bagel-dpo-34b-v0.2         Duisburg      612.578652
                                       Magdeburg     551.448276
                                       München       729.005556
bagel-dpo-34b-v0.2-Yi-34B-Chat         Duisburg      957.491071
                                       Magdeburg     926.598291
                                       München      1321.503876
bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2  Duisburg      809.328571
                                       Magdeburg     714.417323
                                       München      1039.206349
Name: answer_price, dtype: float64
Standard Deviation:  model_combination                      city     
Yi-34B-Chat-Yi-34B-Chat                Duisburg     530.655665
 

In [27]:
print("Mean values: ", agreed_df.groupby(["city"])["answer_price"].mean())
print("Standard Deviation: ", agreed_df.groupby(["city"])["answer_price"].std())

Mean values:  city
Duisburg      812.088468
Magdeburg     717.469419
München      1008.285032
Name: answer_price, dtype: float64
Standard Deviation:  city
Duisburg     360.719839
Magdeburg    213.598968
München      365.555194
Name: answer_price, dtype: float64


In [28]:
agreed_df["answer_likability"] = agreed_df["answer_likability"].astype(str)
agreed_df["answer_likability"] = agreed_df["answer_likability"].str.replace(".0", "")
agreed_df = agreed_df[agreed_df["answer_likability"] != "Null"]
agreed_df["answer_likability"] = agreed_df["answer_likability"].astype(int)
agreed_df["answer_likability"].value_counts()

answer_likability
7     644
8     602
10    329
9     240
1      24
6       9
5       8
3       5
2       2
Name: count, dtype: int64

In [48]:
agreed_df.head()

,Unnamed: 0,answer_name,answer_agreement,answer_price,answer_likability,answer_discrimination,answer_text_discrimination,city,model_landlord,model_renter,renter_name,experiment_id,interview_partner,answer_conversation_partner,correct_character,model_combination
0,0,Peter Schmidt,Yes,740,10,No,NaN,Duisburg,bagel-dpo-34b-v0.2,bagel-dpo-34b-v0.2,Emilia Müller,22963780-cd3b-453f-afb4-b8d6abdd09ea,landlord,Emilia Müller,1.0,bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2
1,1,Emilia Müller,Yes,740,10,No,NaN,Duisburg,bagel-dpo-34b-v0.2,bagel-dpo-34b-v0.2,Emilia Müller,22963780-cd3b-453f-afb4-b8d6abdd09ea,renter,Peter Schmidt,1.0,bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2
2,2,Peter Schmidt,Yes,875,8,No,NaN,Duisburg,bagel-dpo-34b-v0.2,bagel-dpo-34b-v0.2,Emilia Müller,2b711fd6-d419-44e7-a1e2-b6bd89ce9274,landlord,Emilia Müller,1.0,bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2
3,3,Emilia Müller,Yes,875,9,No,NaN,Duisburg,bagel-dpo-34b-v0.2,bagel-dpo-34b-v0.2,Emilia Müller,2b711fd6-d419-44e7-a1e2-b6bd89ce9274,renter,Peter Schmidt,1.0,bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2
4,4,Peter Schmidt,Yes,900,7,No,NaN,Duisburg,bagel-dpo-34b-v0.2,bagel-dpo-34b-v0.2,Emilia Müller,317c8997-7846-4411-8840-ba0136107a13,landlord,Emilia Müller,1.0,bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2


In [51]:
print("Correlation values: ", agreed_df.groupby(["model_combination"])[["answer_likability", "answer_price"]].corr(method = "pearson"))
print("Correlation values: ", agreed_df.groupby(["interview_partner"])[["answer_likability", "answer_price"]].corr(method = "pearson"))

Correlation values:                                                           answer_likability  \
model_combination                                                            
Yi-34B-Chat-Yi-34B-Chat               answer_likability           1.000000   
                                      answer_price                0.006964   
Yi-34B-Chat-bagel-dpo-34b-v0.2        answer_likability           1.000000   
                                      answer_price               -0.085615   
bagel-dpo-34b-v0.2-Yi-34B-Chat        answer_likability           1.000000   
                                      answer_price                0.102677   
bagel-dpo-34b-v0.2-bagel-dpo-34b-v0.2 answer_likability           1.000000   
                                      answer_price                0.012472   

                                                         answer_price  
model_combination                                                      
Yi-34B-Chat-Yi-34B-Chat               answer_likabilit

In [53]:
print("Correlation values: ", agreed_df.groupby(["model_combination","interview_partner"])[["answer_likability", "answer_price"]].corr(method = "pearson"))

Correlation values:                                                                             answer_likability  \
model_combination                     interview_partner                                        
Yi-34B-Chat-Yi-34B-Chat               landlord          answer_likability           1.000000   
                                                        answer_price                0.009317   
                                      renter            answer_likability           1.000000   
                                                        answer_price                0.006271   
Yi-34B-Chat-bagel-dpo-34b-v0.2        landlord          answer_likability           1.000000   
                                                        answer_price               -0.034588   
                                      renter            answer_likability           1.000000   
                                                        answer_price               -0.146192   
bagel-dpo-34b-v0.2-

In [54]:
rho = agreed_df.groupby(["interview_partner"])[["answer_likability", "answer_price"]].corr()
pval = agreed_df.groupby(["interview_partner"])[["answer_likability", "answer_price"]].corr(method=lambda x, y: pearsonr(x, y)[1]) - np.eye(*rho.shape)
p = pval.applymap(lambda x: ''.join(['*' for t in [.05, .01, .001] if x<=t]))
rho.round(2).astype(str) + p

answer_likability answer_price
interview_partner                                                 
landlord          answer_likability            1.0***        0.1**
                  answer_price                  0.1**       1.0***
renter            answer_likability               1.0     -0.13***
                  answer_price               -0.13***          1.0